# Page

## Architecture

### Overall Page

**Important:**
- lp (line pointer) - an object, points to the physical address of the tuple in the page and contains some information about it.
- ctid (tuple id pointer) - a pair of (page_num, lp_num), points to the lp of the tuple.
- tuple (row here) - actual data payload.

**Not in scope:**
- special space - a reserved place for index access methods data to store anything they need (empty on heap).

<img src="./helpers/block-layout.png" alt="drawing" width="600"/>

### Line Pointer

[Source Code](https://github.com/postgres/postgres/blob/ae482a7ec521e09bb0dbfc261da6e6170a5ac29f/src/include/storage/itemid.h#L25):

```C
typedef struct ItemIdData
{
	unsigned	lp_off:15,		/* offset to tuple (from start of page) */
			lp_flags:2,		/* state of line pointer, changed mainly by VACUUM */
			lp_len:15;		/* byte length of tuple */
} ItemIdData;
```

## Hands On

### Prepare ENV

```SQL
DROP TABLE IF EXISTS tbl;
SELECT generate_series AS id, 0 AS data
INTO tbl
FROM generate_series(1,10);

/*** (detailed in VACUUM module) ***/
-- Turn autovacuum off
ALTER TABLE tbl SET (autovacuum_enabled = false);

-- Look at the page layout
DROP VIEW IF EXISTS vw_inspect_tbl_first_page;
CREATE VIEW vw_inspect_tbl_first_page AS
SELECT
	lp AS line_pointer
	,lp_off AS liner_pointer_offset
	,lp_len AS liner_pointer_length
	,lp_off - lp_len AS next_liner_pointer_offset
	,t_ctid
FROM
	heap_page_items(get_raw_page('tbl', 0)) hp;
```

### Page layout

```SQL
SELECT * FROM vw_inspect_tbl_first_page;

SELECT 
	lower AS line_pointers_offset
	,upper AS data_offset
	,upper - lower free_space
FROM page_header(get_raw_page('tbl', 0));
```

### Update

```SQL
-- Single row
UPDATE tbl SET data = 1 WHERE id = 2;
SELECT * FROM vw_inspect_tbl_first_page;
```

```SQL
-- Multiple rows
UPDATE tbl SET data = 1;
SELECT * FROM vw_inspect_tbl_first_page;
```

## Delete

```SQL
-- Single row
DELETE FROM tbl WHERE id = 1;
```

```SQL
-- Multiple rows
DELETE FROM tbl WHERE 1 = 1;
```